#Food Vision 10 Classes 10% Data Classification Model

##Importing Libraries and Data

In [ ]:
 import zipfile
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zipRef = zipfile.ZipFile('10_food_classes_10_percent.zip')
zipRef.extractall()
zipRef.close()

--2022-10-12 05:07:23--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.128, 74.125.197.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   270MB/s    in 0.6s    

2022-10-12 05:07:24 (270 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [ ]:
import os
for dirpath, dirnames, filenames in os.walk('10_food_classes_10_percent'):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'")

There are 2 directories and 0 images in '10_food_classes_10_percent'
There are 10 directories and 0 images in '10_food_classes_10_percent/test'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice

##Converting Data to Batches

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

imageSize = (224, 224)
batchSize = 32

trainDir = '10_food_classes_10_percent/train/'
testDir = '10_food_classes_10_percent/test/'

dataGen = ImageDataGenerator(rescale = 1./255)

trainDataGen = dataGen.flow_from_directory(trainDir,
                                           target_size=imageSize,
                                           batch_size=batchSize,
                                           class_mode='categorical')

testDataGen = dataGen.flow_from_directory(testDir,
                                          target_size=imageSize,
                                          batch_size=batchSize,
                                          class_mode='categorical')

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import datetime

def tbCallback(dirname, expName):
  logdir = dirname + "/" + expName + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tbCallbacks = tf.keras.callbacks.TensorBoard(log_dir=logdir)
  print(f"Saving Tensorboard files to {logdir}")
  return tbCallbacks

In [ ]:
resnetLink = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"
effnetLink = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [ ]:
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
def createModel(url, imSize, num_classes = 10):
  featureExtractor = hub.KerasLayer(url, trainable=False, name="featureExtrator", input_shape=imSize+(3,))

  model = tf.keras.Sequential([
      featureExtractor,
      layers.Dense(num_classes, activation='softmax', name="outputLayer")
  ])

  return model
  

In [ ]:
trainDataGen.num_classes

10

##Using Pretrained Resnet Model

In [ ]:
resnetModel = createModel(resnetLink, imageSize, num_classes = trainDataGen.num_classes)

In [ ]:
resnetModel.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 featureExtrator (KerasLayer  (None, 2048)             23564800  
 )                                                               
                                                                 
 outputLayer (Dense)         (None, 10)                20490     
                                                                 
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


##Compiling the model

In [ ]:
resnetModel.compile(loss="categorical_crossentropy",
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])

##Training for 5 Epochs

In [ ]:
history1 = resnetModel.fit(trainDataGen, epochs = 5, steps_per_epoch=len(trainDataGen), validation_data=testDataGen, validation_steps=len(testDataGen), 
                callbacks=[tbCallback(dirname = "tensorflor_hub", expName= "Resnet_50_v2")])

Saving Tensorboard files to tensorflor_hub/Resnet_50_v2/20221012-052938
Epoch 1/5
24/24 [==============================] - 20s 708ms/step - loss: 1.9783 - accuracy: 0.3147 - val_loss: 1.1922 - val_accuracy: 0.6264
Epoch 2/5
24/24 [==============================] - 16s 675ms/step - loss: 0.9299 - accuracy: 0.7307 - val_loss: 0.8305 - val_accuracy: 0.7344
Epoch 3/5
24/24 [==============================] - 16s 671ms/step - loss: 0.6220 - accuracy: 0.8293 - val_loss: 0.7323 - val_accuracy: 0.7620
Epoch 4/5
24/24 [==============================] - 16s 675ms/step - loss: 0.4757 - accuracy: 0.8813 - val_loss: 0.6882 - val_accuracy: 0.7748
Epoch 5/5
24/24 [==============================] - 16s 674ms/step - loss: 0.3966 - accuracy: 0.9000 - val_loss: 0.6755 - val_accuracy: 0.7832


##Creating Function to plot the trained model performance

In [ ]:
from bokeh.plotting import figure, show
# Our main plotting package (must have explicit import of submodules)
import bokeh.io
import bokeh.plotting
import numpy as np

# Enable viewing Bokeh plots in the notebook
bokeh.io.output_notebook()

def plotLoss(modelHist):
  fig = figure(title="Loss Vs Epoch", x_axis_label="Epoch", y_axis_label="Loss")
  fig.line(np.array(range(1, len(modelHist.history['loss'])+1)), modelHist.history['loss'], legend_label='Training Loss', line_width=2, color='blue')
  fig.line(np.array(range(1, len(modelHist.history['loss'])+1)), modelHist.history['val_loss'], legend_label='Validation Loss', line_width=2, color='red')
  show(fig)

def plotAccuracy(modelHist):
  fig = figure(title="Accuracy Vs Epoch", x_axis_label="Epoch", y_axis_label="Accuracy")
  fig.line(np.array(range(1, len(modelHist.history['accuracy'])+1)), modelHist.history['accuracy'], legend_label='Training Accuracy', line_width=2, color='green')
  fig.line(np.array(range(1, len(modelHist.history['accuracy'])+1)), modelHist.history['val_accuracy'], legend_label='Validation Accuracy', line_width=2, color='purple')
  show(fig)




In [ ]:
plotLoss(history1)

In [ ]:
plotAccuracy(history1)

Insight

Clearly the model is overfitting. We can try Efficient Net model and check the accuracy.

##Creating Efficient Net Model

In [ ]:
effnetModel = createModel(effnetLink, imageSize, num_classes = trainDataGen.num_classes)

In [ ]:
effnetModel.compile(loss="categorical_crossentropy",
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])

##Training for 5 Epochs

In [ ]:
history2 = effnetModel.fit(trainDataGen, epochs=5, steps_per_epoch=len(trainDataGen), validation_data=testDataGen, validation_steps=len(testDataGen), 
                           callbacks=[tbCallback(dirname = "tensorflor_hub", expName= "Effnet")])

Saving Tensorboard files to tensorflor_hub/Effnet/20221012-053111
Epoch 1/5
24/24 [==============================] - 24s 683ms/step - loss: 1.7925 - accuracy: 0.4840 - val_loss: 1.2765 - val_accuracy: 0.7392
Epoch 2/5
24/24 [==============================] - 15s 637ms/step - loss: 1.0508 - accuracy: 0.7773 - val_loss: 0.8582 - val_accuracy: 0.8256
Epoch 3/5
24/24 [==============================] - 15s 642ms/step - loss: 0.7505 - accuracy: 0.8427 - val_loss: 0.6904 - val_accuracy: 0.8500
Epoch 4/5
24/24 [==============================] - 15s 637ms/step - loss: 0.6050 - accuracy: 0.8653 - val_loss: 0.6075 - val_accuracy: 0.8560
Epoch 5/5
24/24 [==============================] - 15s 638ms/step - loss: 0.5121 - accuracy: 0.8853 - val_loss: 0.5542 - val_accuracy: 0.8644


##Plotting the accuracy

In [ ]:
plotLoss(history2)

In [ ]:
plotAccuracy(history2)

The Efficient Net does better than resnet. 

##Saving the model to tensorboard for more visualisation.

In [ ]:
!tensorboard dev upload --logdir ./tensorflor_hub/ \
  --name "Effnet vs Resnet" \
  --description "Comnparing two models" \
  --one_shot 


New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/GflrercbRFWaou4LgRsQAQ/

[2022-10-12T05:32:44] Started scanning logdir.
[2022-10-12T05:32:46] Total uploaded: 60 scalars, 0 tensors, 2 binary objects (5.5 MB)
[2022-10-12T05:32:46] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/GflrercbRFWaou4LgRsQAQ/
